In [3]:
import os
import argparse

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Define the required scopes
SCOPES = ["https://www.googleapis.com/auth/tasks.readonly"]

def authenticate():
    """Authenticate user and return Google Tasks service object."""
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)

        # Save credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("tasks", "v1", credentials=creds)
        return service
    except HttpError as err:
        print(f"An error occurred: {err}")
        return None

def list_task_lists(service, max_results=10):
    """List task lists from the user's Google Tasks account."""
    try:
        results = service.tasklists().list(maxResults=max_results).execute()
        items = results.get("items", [])

        if not items:
            print("No task lists found.")
            return

        print("Task lists:")
        for item in items:
            print(f"- {item['title']} ({item['id']})")

    except HttpError as err:
        print(f"API error: {err}")

def main():
    parser = argparse.ArgumentParser(description="CLI app to access Google Tasks")
    parser.add_argument(
        "--list",
        action="store_true",
        help="List the user's task lists from Google Tasks",
    )
    parser.add_argument(
        "--max",
        type=int,
        default=10,
        help="Maximum number of task lists to retrieve (default: 10)"
    )
    args = parser.parse_args()

    service = authenticate()
    if not service:
        return

    if args.list:
        list_task_lists(service, args.max)
    else:
        parser.print_help()

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--list] [--max MAX]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/adi/Library/Jupyter/runtime/kernel-v32dff074c61c7588f5949ee90abf7669d153161d3.json


SystemExit: 2

/opt/anaconda3/envs/habit/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
